In [10]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import re
import numpy as np

chrome_options = Options()
chrome_options.headless = True  # Run Chrome in headless mode (no GUI)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("disable-infobars")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

 # Setup Edge WebDriver
driver_service = Service(r"C:\\Users\\tragh\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")  # Update path to your chromedriver
driver = webdriver.Chrome(service=driver_service, options=chrome_options)


# IMDb Watchlist URL
url = "https://www.imdb.com/list/ls596927598/?page=1"
driver.get(url)

time.sleep(5)  # Wait for page to load

# Parse watchlist page
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Extract item URLs
URL = []

for i in soup.find_all("div",class_="ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-143a3ae8-2 ezwMlU dli-title with-margin"):
    a_tag = i.find("a")
    if a_tag and a_tag.get("href"):
        full_url = f"https://www.imdb.com{a_tag.get('href')}"
        URL.append(full_url)


In [31]:
len(URL)

250

In [20]:
Title = [] #working
Year=[]  #working
Rating = [] #working
Directors = [] #working
stars_list = []
Votes = []
Languages = []
Duration  = []
Gross = []
production_names = []
Genres = []


URL = URL[:10]

for i in URL:
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    response=requests.get(i,headers=request_header)
    print(response)
    soup=BeautifulSoup(response.text)
    
    for j in soup.find_all("span", class_ = 'hero__primary-text'):
        title=j.text
        if title:
            Title.append(title)
        else:
            Title.append(np.nan)

    for k in soup.find_all("a", class_ = 'ipc-link ipc-link--baseAlt ipc-link--inherit-color'):
        text=k.text
        
        year=re.findall(r"^\d{4}",text)
        if year:
            Year.append(year[0])

    for z in soup.find_all('div', class_ = 'sc-3b24bae4-11 iDIjTq'): #writers also
        text = z.text
        
       
        rat = re.findall(r"(\d\.\d)/10",text)   
        if rat:
            Rating.append(rat[0])
        else:
            Rating.append(np.nan)

        director = re.findall(r"Director[s]?(.*?)(Writer[s]?)", text)
        if director:
            director_names = director[0][0].strip()
            Directors.append(director_names)
        else:
            Directors.append(np.nan)

        votes = re.findall(r"10([\d.,KkMm]+)YOUR", text)   
        if votes:
            Votes.append(votes[0])
        else:
            Votes.append(np.nan)

        match = re.search(r"Stars(.*?)(?=\d+User reviews)", text)
        if match:
            raw_names = match.group(1).strip()
        # Split before each capitalized first name except the first
            names = re.findall(r'[A-Z][a-z]+(?:\s[A-Z][a-z]+)+', raw_names)
            stars_list.append(','.join(names))
        else:
            stars_list.append(np.nan)

    for d in soup.find_all('ul', class_='ipc-inline-list ipc-inline-list--show-dividers sc-103e4e3c-2 cMcwpt baseAlt baseAlt'):
        text = d.text.strip()
        
        match = re.findall(r"\d{1}h\s\d{1,2}m", text)
        Duration.append(match[0] if match else np.nan)

        

    genre_div = soup.find('div', class_='ipc-chip-list--baseAlt ipc-chip-list ipc-chip-list--nowrap sc-865706aa-4 bjiBzC')
    if genre_div:
        genre_tags = genre_div.find_all('a')
        genre_names = [g.text.strip() for g in genre_tags]
        Genres.append(', '.join(genre_names))
    else:
        Genres.append(np.nan)


    language_section = soup.find('li', attrs={'data-testid': 'title-details-languages'})

    if language_section:
        langs = [lang.text.strip() for lang in language_section.find_all('a')]
        Languages.append(", ".join(langs))  
    else:
        Languages.append(np.nan)

    
    gross_found = False
    for li in soup.find_all('li', class_='ipc-metadata-list__item ipc-metadata-list__item--align-end sc-a3d75170-2 eyloUU'):
        text = li.text.strip()
        match = re.search(r'Gross worldwide\$(\d[\d,]*)', text)
        if match:
            Gross.append(match.group(1))
            gross_found = True
            break
    if not gross_found:
        Gross.append(np.nan)

    for i in soup.find_all('ul', class_='ipc-metadata-list ipc-metadata-list--dividers-all ipc-metadata-list--base'):
        text = i.text.strip()
        match = re.findall(r'Production compan(?:y|ies)(.*?)(?=See)', text)
        if match:
            cleaned = re.sub(r'(?<=[a-z])(?=[A-Z])', ', ', match[0].strip())
            production_names.append(cleaned)
        else:
            continue 

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [22]:
Genres

['Action Epic, One-Person Army Action, Action, Drama, Thriller',
 'Action Epic, Costume Drama, Epic, Historical Epic, One-Person Army Action, Period Drama, Tragedy, Action, Drama, History',
 'Action, Drama, Thriller',
 'Legal Drama, Drama',
 'Cop Drama, One-Person Army Action, Police Procedural, Psychological Thriller, Suspense Mystery, Whodunnit, Action, Crime, Drama, Mystery',
 'Docudrama, Legal Drama, Period Drama, Biography, Drama, History',
 'Coming-of-Age, Psychological Drama, Comedy, Drama, Romance',
 'Action Epic, One-Person Army Action, Action, Crime, Drama, Thriller',
 'Action Epic, One-Person Army Action, Tragedy, Action, Drama, Thriller',
 'Caper, Crime, Drama, Thriller']

In [16]:
Title = [] #working
Year=[]  #working
Rating = [] #working
Directors = [] #working
stars_list = []
Votes = []
Languages = []
Duration  = []
Gross = []
production_names = []
Genres = []


URL = URL[:25]

for i in URL:
    request_header = {'Content-Type': 'text/html; charset=UTF-8','User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0','Accept-Encoding': 'gzip, deflate, br'}
    response=requests.get(i,headers=request_header)
    print(response)
    soup=BeautifulSoup(response.text)
    
    
        

    genre_div = soup.find('div', class_='ipc-chip-list--baseAlt ipc-chip-list ipc-chip-list--nowrap sc-865706aa-4 bjiBzC')
    if genre_div:
        genre_tags = genre_div.find_all('a')
        genre_names = [g.text.strip() for g in genre_tags]
        Genres.append(', '.join(genre_names))
    else:
        Genres.append(np.nan)




<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [18]:
Genres

['Action Epic, One-Person Army Action, Action, Drama, Thriller',
 'Action Epic, Costume Drama, Epic, Historical Epic, One-Person Army Action, Period Drama, Tragedy, Action, Drama, History',
 'Action, Drama, Thriller',
 'Legal Drama, Drama',
 'Cop Drama, One-Person Army Action, Police Procedural, Psychological Thriller, Suspense Mystery, Whodunnit, Action, Crime, Drama, Mystery',
 'Docudrama, Legal Drama, Period Drama, Biography, Drama, History',
 'Coming-of-Age, Psychological Drama, Comedy, Drama, Romance',
 'Action Epic, One-Person Army Action, Action, Crime, Drama, Thriller',
 'Action Epic, One-Person Army Action, Tragedy, Action, Drama, Thriller',
 'Caper, Crime, Drama, Thriller',
 'Legal Drama, Political Drama, Political Thriller, Drama, Thriller',
 'Horror',
 'Folk Horror, Supernatural Horror, Drama, Horror, Thriller',
 'Docudrama, Biography, Drama',
 'Conspiracy Thriller, Thriller',
 'Comedy',
 'Quirky Comedy, Comedy',
 'Buddy Comedy, Coming-of-Age, Quirky Comedy, Comedy, Drama',

In [38]:
Title = []
Year = []
Rating = []
Directors = []
stars_list = []
Votes = []
Languages = []
Duration = []
Gross = []
production_names = []
Genres = []



for i in URL:
    request_header = {
        'Content-Type': 'text/html; charset=UTF-8',
        'User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0',
        'Accept-Encoding': 'gzip, deflate, br'
    }
    response = requests.get(i, headers=request_header)
    print(response)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Title
    j = soup.find("span", class_='hero__primary-text')
    Title.append(j.text.strip() if j else np.nan)

    # Year
    year_found = False
    for k in soup.find_all("a", class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color'):
        text = k.text
        year = re.findall(r"^\d{4}", text)
        if year:
            Year.append(year[0])
            year_found = True
            break
    if not year_found:
        Year.append(np.nan)

    # Rating, Directors, Votes, Stars
    z = soup.find('div', class_='sc-3b24bae4-11 iDIjTq')
    if z:
        text = z.text

        rat = re.findall(r"(\d\.\d)/10", text)
        Rating.append(rat[0] if rat else np.nan)

        director = re.findall(r"Director[s]?(.*?)(Writer[s]?)", text)
        Directors.append(director[0][0].strip() if director else np.nan)

        votes = re.findall(r"10([\d.,KkMm]+)YOUR", text)
        Votes.append(votes[0] if votes else np.nan)

        match = re.search(r"Stars(.*?)(?=\d+User reviews)", text)
        if match:
            raw_names = match.group(1).strip()
            names = re.findall(r'[A-Z][a-z]+(?:\s[A-Z][a-z]+)+', raw_names)
            stars_list.append(', '.join(names))
        else:
            stars_list.append(np.nan)
    else:
        Rating.append(np.nan)
        Directors.append(np.nan)
        Votes.append(np.nan)
        stars_list.append(np.nan)

    # Duration
    d = soup.find('ul', class_='ipc-inline-list ipc-inline-list--show-dividers sc-103e4e3c-2 cMcwpt baseAlt baseAlt')
    if d:
        match = re.findall(r"\d{1}h\s\d{1,2}m", d.text.strip())
        Duration.append(match[0] if match else np.nan)
    else:
        Duration.append(np.nan)

    # Genres
    m = soup.find('div', class_='ipc-chip-list--baseAlt ipc-chip-list ipc-chip-list--nowrap sc-865706aa-4 bjiBzC')
    if m:
        genres = re.findall(r'[A-Z][a-z]+', m.text.strip())
        Genres.append(", ".join(genres) if genres else np.nan)
    else:
        Genres.append(np.nan)

    # Languages
    language_section = soup.find('li', attrs={'data-testid': 'title-details-languages'})
    if language_section:
        langs = [lang.text.strip() for lang in language_section.find_all('a')]
        Languages.append(", ".join(langs))
    else:
        Languages.append(np.nan)

    # Gross (fixed logic)
    gross_found = False
    for li in soup.find_all('li', class_='ipc-metadata-list__item ipc-metadata-list__item--align-end sc-a3d75170-2 eyloUU'):
        text = li.text.strip()
        match = re.search(r'Gross worldwide\$(\d[\d,]*)', text)
        if match:
            Gross.append(match.group(1))
            gross_found = True
            break
    if not gross_found:
        Gross.append(np.nan)

    # Production Companies
    prod_section = soup.find('ul', class_='ipc-metadata-list ipc-metadata-list--dividers-all ipc-metadata-list--base')
    if prod_section:
        text = prod_section.text.strip()
        match = re.findall(r'Production compan(?:y|ies)(.*?)(?=See)', text)
        if match:
            cleaned = re.sub(r'(?<=[a-z])(?=[A-Z])', ', ', match[0].strip())
            production_names.append(cleaned)
        else:
            production_names.append(np.nan)
    else:
        production_names.append(np.nan)



df = pd.DataFrame({
    'Title': Title,
    'Year': Year,
    'Rating': Rating,
    'Directors': Directors,
    'Stars': stars_list,
    'Votes': Votes,
    'Languages': Languages,
    'Duration': Duration,
    'Gross Worldwide': Gross,
    'Production Company': production_names,
    'Genres': Genres
})



<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


KeyboardInterrupt: 

In [225]:
len(Title)

250

In [227]:
len(Year)


250

In [229]:
len(Rating)


250

In [231]:
len(Directors)


250

In [233]:
len(stars_list)


250

In [235]:
len(Votes) 


250

In [239]:
len(Languages) 
 

250

In [241]:
len(Duration) 


250

In [243]:
len(Gross) 


250

In [245]:
len(production_names) 


250

In [247]:
len(Genres)

250

In [249]:
df

,Title,Year,Rating,Directors,Stars,Votes,Languages,Duration,Gross Worldwide,Production Company,Genres
0,Jaat,2025,7.0,Gopichand Malineni,"Sunny Deol, Randeep Hooda, Regena Cassandrra",5.1K,Hindi,2h 38m,"394,257","Mythri Movie Makers, People Media Factory","Action, Epic, One, Person, Army, Action, Actio..."
1,Chhaava,2025,7.4,Laxman Utekar,"Vicky Kaushal, Akshaye Khanna, Rashmika Mandanna",30K,Hindi,2h 41m,"8,219,603",Maddock Films,"Action, Epic, Costume, Drama, Epic, Historical..."
2,Good Bad Ugly,2025,7.2,Adhik Ravichandran,"Ajith Kumar, Trisha Krishnan, Priya Prakash Va...",7.7K,Tamil,2h 19m,"1,348,768",Mythri Movie Makers,"Action, Drama, Thriller"
3,Court - State vs a Nobody,2025,8.0,Ram Jagadeesh,"Priyadarshi Pulikonda, Harsh Roshan, Sri Devi",4.6K,"Telugu, Hindi, Tamil, Kannada, Malayalam",2h 35m,"133,791",Wall Poster Cinema,"Legal, Drama, Drama"
4,Deva,2025,6.8,Rosshan Andrrews,"Pooja Hegde, Shahid Kapoor, Saadhika Syal",24K,Hindi,2h 36m,"1,091,296","Roy Kapur Films, Zee Studios","Cop, Drama, One, Person, Army, Action, Police,..."
...,...,...,...,...,...,...,...,...,...,...,...
245,War,2019,6.6,Siddharth Anand,"Hrithik Roshan, Tiger Shroff, Vaani Kapoor",35K,Hindi,2h 31m,"67,179,155","Reverie Entertainment, Yash Raj Films","Action, Epic, Globetrotting, Adventure, Spy, A..."
246,The Sabarmati Report,2024,6.4,Dheeraj Sarna,"Vikrant Massey, Raashi Khanna, Ridhi Dogra",24K,Hindi,2h 7m,"77,045","Moral of the Story, Balaji Motion Pictures, Ba...","Conspiracy, Thriller, Political, Drama, Politi..."
247,Sangharsh,1999,NaN,NaN,NaN,NaN,Hindi,2h 7m,NaN,NH Studioz,"Psychological, Thriller, Serial, Killer, Actio..."
248,Ground Zero,2025,NaN,Tejas Prabha Vijay Deoskar,NaN,NaN,Hindi,2h 14m,NaN,"Excel Entertainment, Talisman Films","Action, Drama, History"


In [ ]:
df.to_csv("imdbtestmovie.csv", index=False)

In [26]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import numpy as np

# Configure headless Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("disable-infobars")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

# Setup Edge WebDriver (you can replace with ChromeDriverManager if needed)
driver_service = Service(r"C:\\Users\\tragh\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")  # Update this path!
driver = webdriver.Chrome(service=driver_service, options=chrome_options)

# IMDb watchlist pagination
base_url = "https://www.imdb.com/list/ls596927598/?page="
URL = []

# Loop through all 3 pages
for page in range(1, 4):
    full_url = base_url + str(page)
    driver.get(full_url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    for i in soup.find_all("div", class_="ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-143a3ae8-2 ezwMlU dli-title with-margin"):
        a_tag = i.find("a")
        if a_tag and a_tag.get("href"):
            full_url = f"https://www.imdb.com{a_tag.get('href')}"
            URL.append(full_url)


In [28]:
len(URL)

554

In [32]:
# Initialize lists
Title = []
Year = []
Rating = []
Directors = []
stars_list = []
Votes = []
Languages = []
Duration = []
Gross = []
production_names = []
Genres = []

# Scrape each movie page
for i in URL:
    request_header = {
        'Content-Type': 'text/html; charset=UTF-8',
        'User-Agent': 'Chrome/101.0.0.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0',
        'Accept-Encoding': 'gzip, deflate, br'
    }
    response = requests.get(i, headers=request_header)
    print(response)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Title
    j = soup.find("span", class_='hero__primary-text')
    Title.append(j.text.strip() if j else np.nan)

    # Year
    year_found = False
    for k in soup.find_all("a", class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color'):
        year = re.findall(r"^\d{4}", k.text)
        if year:
            Year.append(year[0])
            year_found = True
            break
    if not year_found:
        Year.append(np.nan)

    # Rating, Directors, Votes, Stars
    z = soup.find('div', class_='sc-3b24bae4-11 iDIjTq')
    if z:
        text = z.text
        Rating.append(re.findall(r"(\d\.\d)/10", text)[0] if re.findall(r"(\d\.\d)/10", text) else np.nan)
        director = re.findall(r"Director[s]?(.*?)(Writer[s]?)", text)
        Directors.append(director[0][0].strip() if director else np.nan)
        votes = re.findall(r"10([\d.,KkMm]+)YOUR", text)
        Votes.append(votes[0] if votes else np.nan)
        match = re.search(r"Stars(.*?)(?=\d+User reviews)", text)
        if match:
            raw_names = match.group(1).strip()
            names = re.findall(r'[A-Z][a-z]+(?:\s[A-Z][a-z]+)+', raw_names)
            stars_list.append(', '.join(names))
        else:
            stars_list.append(np.nan)
    else:
        Rating.append(np.nan)
        Directors.append(np.nan)
        Votes.append(np.nan)
        stars_list.append(np.nan)

    # Duration
    d = soup.find('ul', class_='ipc-inline-list ipc-inline-list--show-dividers sc-103e4e3c-2 cMcwpt baseAlt baseAlt')
    Duration.append(re.findall(r"\d{1}h\s\d{1,2}m", d.text.strip())[0] if d and re.findall(r"\d{1}h\s\d{1,2}m", d.text.strip()) else np.nan)

    # Genres
    genre_div = soup.find('div', class_='ipc-chip-list--baseAlt ipc-chip-list ipc-chip-list--nowrap sc-865706aa-4 bjiBzC')
    if genre_div:
        genre_tags = genre_div.find_all('a')
        genre_names = [g.text.strip() for g in genre_tags]
        Genres.append(', '.join(genre_names))
    else:
        Genres.append(np.nan)

    # Languages
    language_section = soup.find('li', attrs={'data-testid': 'title-details-languages'})
    if language_section:
        langs = [lang.text.strip() for lang in language_section.find_all('a')]
        Languages.append(", ".join(langs))
    else:
        Languages.append(np.nan)

    # Gross
    gross_found = False
    for li in soup.find_all('li', class_='ipc-metadata-list__item ipc-metadata-list__item--align-end sc-a3d75170-2 eyloUU'):
        match = re.search(r'Gross worldwide\$(\d[\d,]*)', li.text.strip())
        if match:
            Gross.append(match.group(1))
            gross_found = True
            break
    if not gross_found:
        Gross.append(np.nan)

    # Production Company
    prod_section = soup.find('ul', class_='ipc-metadata-list ipc-metadata-list--dividers-all ipc-metadata-list--base')
    if prod_section:
        match = re.findall(r'Production compan(?:y|ies)(.*?)(?=See)', prod_section.text.strip())
        if match:
            cleaned = re.sub(r'(?<=[a-z])(?=[A-Z])', ', ', match[0].strip())
            production_names.append(cleaned)
        else:
            production_names.append(np.nan)
    else:
        production_names.append(np.nan)


# Store in DataFrame
df = pd.DataFrame({
    'Title': Title,
    'Year': Year,
    'Rating': Rating,
    'Directors': Directors,
    'Stars': stars_list,
    'Votes': Votes,
    'Languages': Languages,
    'Duration': Duration,
    'Gross Worldwide': Gross,
    'Production Company': production_names,
    'Genres': Genres
})

# Output: save or view

# df.to_csv("imdb_watchlist_movies.csv", index=False)

# Quit the browser
driver.quit()


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [34]:
df

,Title,Year,Rating,Directors,Stars,Votes,Languages,Duration,Gross Worldwide,Production Company,Genres
0,Jaat,2025,6.9,Gopichand Malineni,"Sunny Deol, Randeep Hooda, Regena Cassandrra",5.2K,Hindi,2h 38m,"394,257","Mythri Movie Makers, People Media Factory","Action Epic, One-Person Army Action, Action, D..."
1,Chhaava,2025,7.4,Laxman Utekar,"Vicky Kaushal, Akshaye Khanna, Rashmika Mandanna",30K,Hindi,2h 41m,"8,219,603",Maddock Films,"Action Epic, Costume Drama, Epic, Historical E..."
2,Good Bad Ugly,2025,7.2,Adhik Ravichandran,"Ajith Kumar, Trisha Krishnan, Priya Prakash Va...",7.7K,Tamil,2h 19m,"1,348,768",Mythri Movie Makers,"Action, Drama, Thriller"
3,Court - State vs a Nobody,2025,8.0,Ram Jagadeesh,"Priyadarshi Pulikonda, Harsh Roshan, Sri Devi",4.7K,"Telugu, Hindi, Tamil, Kannada, Malayalam",2h 35m,"133,791",Wall Poster Cinema,"Legal Drama, Drama"
4,Deva,2025,6.8,Rosshan Andrrews,"Pooja Hegde, Shahid Kapoor, Saadhika Syal",24K,Hindi,2h 36m,"1,091,296","Roy Kapur Films, Zee Studios","Cop Drama, One-Person Army Action, Police Proc..."
...,...,...,...,...,...,...,...,...,...,...,...
549,Viswasam,2019,6.5,Siva,"Ajith Kumar, Jagapathi Babu",11K,Tamil,2h 33m,"918,734","Sathya Jyothi Films, Goldmines Telefilms","One-Person Army Action, Action, Drama"
550,Annaatthe,2021,4.0,Siva,"Aditya Nath Uppal, Keerthy Suresh",11K,Tamil,2h 43m,"230,333","Sun Pictures, Sun Pictures","Action, Comedy, Drama"
551,Kaala,2018,6.5,Pa. Ranjith,Nana Patekar,11K,Tamil,2h 42m,"1,509,872",Wunderbar Films,"Action, Crime, Drama"
552,Yennai Arindhaal,2015,7.3,Gautham Vasudev Menon,"Ajith Kumar, Arun Vijay, Trisha Krishnan",11K,Tamil,2h 48m,"841,040",Sri Sathya Sai Movies,"Action, Crime, Drama, Thriller"


In [36]:
df.to_csv("Updatedmovies.csv", index=False)

In [259]:
df

,Title,Year,Rating,Directors,Stars,Votes,Languages,Duration,Gross Worldwide,Production Company,Genres
0,Jaat,2025,6.9,Gopichand Malineni,"Sunny Deol, Randeep Hooda, Regena Cassandrra",5.1K,Hindi,2h 38m,"394,257","Mythri Movie Makers, People Media Factory","Action, Epic, One, Person, Army, Action, Actio..."
1,Chhaava,2025,7.4,Laxman Utekar,"Vicky Kaushal, Akshaye Khanna, Rashmika Mandanna",30K,Hindi,2h 41m,"8,219,603",Maddock Films,"Action, Epic, Costume, Drama, Epic, Historical..."
2,Good Bad Ugly,2025,7.2,Adhik Ravichandran,"Ajith Kumar, Trisha Krishnan, Priya Prakash Va...",7.7K,Tamil,2h 19m,"1,348,768",Mythri Movie Makers,"Action, Drama, Thriller"
3,Court - State vs a Nobody,2025,8.0,Ram Jagadeesh,"Priyadarshi Pulikonda, Harsh Roshan, Sri Devi",4.7K,"Telugu, Hindi, Tamil, Kannada, Malayalam",2h 35m,"133,791",Wall Poster Cinema,"Legal, Drama, Drama"
4,Deva,2025,6.8,Rosshan Andrrews,"Pooja Hegde, Shahid Kapoor, Saadhika Syal",24K,Hindi,2h 36m,"1,091,296","Roy Kapur Films, Zee Studios","Cop, Drama, One, Person, Army, Action, Police,..."
...,...,...,...,...,...,...,...,...,...,...,...
549,Viswasam,2019,6.5,Siva,"Ajith Kumar, Jagapathi Babu",11K,Tamil,2h 33m,"918,734","Sathya Jyothi Films, Goldmines Telefilms","One, Person, Army, Action, Action, Drama"
550,Annaatthe,2021,4.0,Siva,"Aditya Nath Uppal, Keerthy Suresh",11K,Tamil,2h 43m,"230,333","Sun Pictures, Sun Pictures","Action, Comedy, Drama"
551,Kaala,2018,6.5,Pa. Ranjith,Nana Patekar,11K,Tamil,2h 42m,"1,509,872",Wunderbar Films,"Action, Crime, Drama"
552,Yennai Arindhaal,2015,7.3,Gautham Vasudev Menon,"Ajith Kumar, Arun Vijay, Trisha Krishnan",11K,Tamil,2h 48m,"841,040",Sri Sathya Sai Movies,"Action, Crime, Drama, Thriller"


In [261]:
df.to_csv("500+movies.csv", index=False)